This notebook uses fraud detection dataset.
We try **AdaBoost** classification with different settings to find the best model.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
df = pd.read_csv("../input/creditcard.csv")
# print(df.describe())
print(df['Class'].value_counts())

y = df['Class']
df = df.drop("Class",axis=1)

Prepare training and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.1, random_state=0)

 Detailed settings are follows.

 1. Various base classifiers: i) decision tree, and ii) Naive Bayes (binomial)
 2. Different number of estimators in {40, 50, 60}
 

To find the best model, we use **grid search**.

In [ ]:
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB


As running grid search with several setting combinations is expensive, we run them in parallel by setting `n_jobs` parameters. 

If we set `n_jobs=-1`, grid search will detect how many cores are installed and uses them all.

In [ ]:
boost_clf = AdaBoostClassifier() 
param_grid = {'n_estimators': np.arange(40, 70, 10), 'base_estimator': [DecisionTreeClassifier(), 
                                                                        GaussianNB()]}
gs_boost_clf = GridSearchCV(boost_clf, param_grid, n_jobs=-1, verbose=1)

In [ ]:
gs_boost_clf = gs_boost_clf.fit(X_train, y_train)
boost_pred = gs_boost_clf.predict(X_test)
print(classification_report(y_test, y_pred=boost_pred))

In [ ]:
print('Best score of AdaBoost classifier: %.2f' %gs_boost_clf.best_score_)
for param_name in param_grid.keys():
    print('%s %r' %(param_name, gs_boost_clf.best_params_[param_name]))